# Fine-Tuning Gemma for 54 Languages
This notebook demonstrates the multilingual fine-tuning of the Gemma model on 54 datasets. We will explore the workflow from data loading and preprocessing to model fine-tuning and evaluation.

**Key Steps:**
1. Setup environment variables for Kaggle and Weights & Biases (wandb).
2. Load and preprocess the dataset.
3. Set up model parallelism for TPU utilization.
4. Fine-tune the Gemma model using LoRA (Low-Rank Adaptation).
5. Evaluate model performance before and after fine-tuning.


##### you can look into the fine-tuning process logs in here: [link](https://wandb.ai/this-is-the-way-2005-independent/fine-tuning-gemma2_2b_Polyglot)

#### Device:
we used the TPU VM v3-8 from kaggle.
#### Base model:
we used gemma2_2b_en

##### Note:
this is my gemma cookbook, I'm uploading all gemma related notebooks:https://github.com/Mhdaw/Gemma2

### Step 0: Installing the Required Libraries and Frameworks
To ensure that all necessary libraries and frameworks are installed, run the following commands:

In [1]:
!pip install -q -U keras-nlp keras datasets kagglehub keras_hub 
!pip install -q -U tensorflow-text
# Install tensorflow-cpu so tensorflow does not attempt to access the TPU.
!pip install -q -U tensorflow-cpu
!pip install -q -U wandb


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.16.0 requires tensorflow<2.17,>=2.16, but you have tensorflow 2.18.0 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import jax
jax.devices()

E0000 00:00:1736031878.027382      74 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/common_lib.cc:479


E0104 23:04:38.072993633      74 oauth2_credentials.cc:238]            oauth_fetch: UNKNOWN:C-ares status is not ARES_SUCCESS qtype=A name=metadata.google.internal. is_balancer=0: Domain name not found {grpc_status:2, created_time:"2025-01-04T23:04:38.072977498+00:00"}


[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0),
 TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1),
 TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0),
 TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1),
 TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0),
 TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1),
 TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0),
 TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]

## Step 1: Setup Environment Variables
We will configure the environment variables required for:
- Kaggle API access
- Weights & Biases for tracking experiments
- TensorFlow backend optimization.


In [3]:
import os
# Set the environment variables for Kaggle and Weights & Biases.
# from kaggle_secrets import UserSecretsClient
# from google.colab import userdata
#import getpass
os.environ["KAGGLE_USERNAME"] = "your-username"# or UserSecretsClient().get_secret(KAGGLE_USERNAME) or userdata.get(KAGGLE_USERNAME) or getpass.getpass("Enter your KAGGLE_USERNAME: ")
os.environ["KAGGLE_KEY"] = "kaggle-api-key" # or UserSecretsClient().get_secret(KAGGLE_KEY) or userdata.get(KAGGLE_KEY) or getpass.getpass("Enter your  KAGGLE_KEY: ")
os.environ["WANDB_API_KEY"] = "wand-api-key" # or UserSecretsClient().get_secret(WANDB_API_KEY) or userdata.get(WANDB_API_KEY) or getpass.getpass("Enter your WANDB_API_KEY: ")
os.environ["KERAS_BACKEND"] = "jax"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.00"

In [5]:
import tensorflow as tf
import keras
import keras_nlp
from datasets import load_dataset
import itertools
import wandb
from wandb.integration.keras import WandbMetricsLogger
import matplotlib.pyplot as plt

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step 2: Load and Explore the Dataset
We are using the `allenai/c4` dataset. The dataset is loaded in streaming mode for efficient handling of large datasets.

**Subtasks:**
- Load training and validation datasets for each language.
- Extract sample data for exploration.
- Limit dataset size for efficient experimentation.
- concatenate them and make a general dataset.


Since we want to fine-tune the Gemma 2 9b model for adapting to the 54 languages, we need a good amount of high-quality multilingual text corpus. For that, we use the 'C4' dataset, which is a multilingual text dataset.

You can look into it on Hugging Face: [Link](https://huggingface.co/datasets/allenai/c4)  

**Dataset Summary (from the original dataset page):**  
A colossal, cleaned version of Common Crawl's web crawl corpus. Based on the Common Crawl dataset: [https://commoncrawl.org](https://commoncrawl.org).

This is the processed version of Google's C4 dataset.

In [6]:
languages = ["en","ar","zh","nl","fr","de","it","ja","ko","pl","pt","ru","es","tr","bg",
             "ca","co","cs","da","fi","el","iw","hi","hu","no","ro","sk","sl","sv","vi",
             "fa","af","bn","et","is","th","fil","lt","uk","mr","sw","ta","sq","hy","az",
             "lo","ne","ka","kn","ml","te","ky","pa","gu",]
# we use 54 languages

max_examples_per_lang = 500

In [7]:
all_lang_data = {}

for lang in languages:
    try:
        #print(f"Loading data for {lang}...")
        lang_data = load_dataset("allenai/c4", lang, streaming=True)

        train_text_data = [
            example["text"]
            for example in itertools.islice(lang_data["train"], max_examples_per_lang)
        ]
        val_text_data = [
            example["text"]
            for example in itertools.islice(lang_data["validation"], 75) #Keep validation smaller
        ]

        all_lang_data[lang] = {"train": train_text_data, "validation": val_text_data}
        #print(f"Loaded {len(train_text_data)} training examples for {lang}")

    except Exception as e:  # Catch potential errors (e.g., unsupported language)
        print(f"Error loading data for {lang}: {e}")
        continue # Skip to the next language
print("Data loaded")

Data loaded


In [8]:
full_data_train = []
full_data_val = []
for lang, data in all_lang_data.items():
    full_data_train.extend(data['train'])
    full_data_val.extend(data['validation'])
    #print(f"Loaded {len(data['train'])} train examples and {len(data['validation'])} validation examples for {lang}")
    
print(f"train data length:{len(full_data_train)}")
print(f"validation data length:{len(full_data_val)}")

train data length:27000
validation data length:4050


## Step 3: Data Preprocessing
The text data will be converted into TensorFlow datasets for training and validation. Key preprocessing steps include:
- Creating TensorFlow datasets from plain-text lists.
- Shuffling and batching training data for optimized input.

In [9]:
batch_size = 4

# Convert the lists of text data to TensorFlow datasets
train_data = tf.data.Dataset.from_tensor_slices(full_data_train)
val_data = tf.data.Dataset.from_tensor_slices(full_data_val)

# Preprocess each text sample
def preprocess_text(text):
    return tf.convert_to_tensor(text, dtype=tf.string)

# Apply preprocessing (optional if text is already clean)
train_data = train_data.map(preprocess_text)
val_data = val_data.map(preprocess_text)

# Shuffle and batch the training data
train_data = train_data.shuffle(buffer_size=1000).batch(batch_size)
val_data = val_data.batch(batch_size)

## Step 4: Model Parallelism for Efficient Training
We configure model parallelism using TPUs to handle the large-scale Gemma model. Key components:
- **Device Mesh:** A mapping of TPU devices.
- **Layout Map:** Specifies the sharding strategy for different layers.
- and then we load the model.

## Step 5: Model Overview
We initialize the Gemma model and explore its architecture.

### Key Model Parameters:
- **Model ID:** Pretrained Gemma version for transfer learning.
- **LoRA:** Enable Low-Rank Adaptation for fine-tuning.
- **Sequence Length:** Adjusted for task requirements.


In [10]:
# Create a device mesh with (1, 8) shape so that the weights are sharded across
# all 8 TPUs.
device_mesh = keras.distribution.DeviceMesh(
    (1, 8),
    ["batch", "model"],
    devices=keras.distribution.list_devices(),
)

model_dim = "model"

layout_map = keras.distribution.LayoutMap(device_mesh)

# Assuming the second dimension (2304) of the attention kernels is divisible by 8,
# we shard along that dimension:
layout_map["token_embedding/embeddings"] = (model_dim, None)  # Shard embeddings along model dimension
layout_map["decoder_block.*attention.*(query|key|value)/kernel"] = (None, model_dim, None) # Shard attention kernels along second dimension
layout_map["decoder_block.*attention_output/kernel"] = (None, model_dim, None)  # Shard attention output kernels along second dimension
layout_map["decoder_block.*ffw_gating.*/kernel"] = (None, model_dim)  # Shard FFN gating kernels along second dimension
layout_map["decoder_block.*ffw_linear/kernel"] = (model_dim, None) # Shard FFN linear kernels along first dimension

model_parallel = keras.distribution.ModelParallel(
    layout_map=layout_map,
    batch_dim_name="batch",
)

keras.distribution.set_distribution(model_parallel)
model_id = "/kaggle/input/gemma2/keras/gemma2_2b_en/2" # change this if you want
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset(model_id)
gemma_lm.summary()

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,614,341,888 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,614,341,888 (9.74 GB)

 Trainable params: 2,614,341,888 (9.74 GB)

 Non-trainable params: 0 (0.00 B)

In [12]:
template = "Instruction:\n{instruction}\n\nResponse:\n{response}"

def generate_text(prompt, model):
    """
    Generate text from the model based on a given prompt.
    """
    sampler = keras_nlp.samplers.TopKSampler(k=15, seed=2)
    model.compile(sampler=sampler)
    output = model.generate(prompt, max_length=512)
    return output

## Step 6: Evaluate Model Performance Before Fine-Tuning
Before training, test the model on a set of prompts to benchmark its initial performance. This helps us compare improvements after fine-tuning.

*since we can't test for all languages, we use a subset of them*


In [13]:
test_prompts_multilingual = [
    # English (American)
    "What's a fun fact about the Grand Canyon?",  # US English
    "Write a short story about a talking dog.", # US English
    # Arabic (Modern Standard)
    "ما هي أقدم مدينة في العالم؟", # What is the oldest city in the world?
    "اكتب جملة عن أهمية القراءة.", # Write a sentence about the importance of reading.
    # Chinese (Simplified)
    "长城有多长？", # How long is the Great Wall?
    "写一个关于猫的短篇故事。", # Write a short story about a cat.
    # Dutch
    "Wat is de hoofdstad van Nederland?", # What is the capital of the Netherlands?
    "Schrijf een korte beschrijving van een molen.", # Write a short description of a windmill.
    # French (European)
    "Quelle est la capitale de la France ?", # What is the capital of France?
    "Écris une courte description de la Tour Eiffel.", # Write a short description of the Eiffel Tower.
    # German
    "Was ist die Hauptstadt von Deutschland?", # What is the capital of Germany?
    "Schreibe eine kurze Beschreibung des Brandenburger Tors.", # Write a short description of the Brandenburg Gate.
    # Italian
    "Qual è la capitale d'Italia?", # What is the capital of Italy?
    "Scrivi una breve descrizione del Colosseo.", # Write a short description of the Colosseum.
    # Japanese
    "日本の首都はどこですか？", # What is the capital of Japan?
    "桜について短い文章を書いてください。", # Please write a short sentence about cherry blossoms.
    # Korean
    "한국의 수도는 어디입니까?", # What is the capital of Korea?
    "한국 음식에 대해 간단히 설명해 주세요.", # Please briefly explain about Korean food.
    # Polish
    "Jaka jest stolica Polski?", # What is the capital of Poland?
    "Napisz krótkie opowiadanie o smoku.", # Write a short story about a dragon.
    # Portuguese (Brazilian)
    "Qual é a capital do Brasil?", # What is the capital of Brazil?
    "Escreva uma breve descrição do Cristo Redentor.", # Write a short description of Christ the Redeemer.
    # Russian
    "Какая столица России?", # What is the capital of Russia?
    "Напишите короткий рассказ о медведе.", # Write a short story about a bear.
    # Spanish (European)
    "¿Cuál es la capital de España?", # What is the capital of Spain?
    "Escribe una breve descripción de la Sagrada Familia.", # Write a short description of the Sagrada Familia.
    # Thai
    "ประเทศไทยมีเมืองหลวงชื่ออะไร", # What is the capital of Thailand?
    "เขียนประโยคสั้นๆ เกี่ยวกับวัดไทย", # Write a short sentence about Thai temples.
    # Turkish
    "Türkiye'nin başkenti neresidir?", # What is the capital of Turkey?
    "Kapadokya hakkında kısa bir açıklama yazın.", # Write a short description about Cappadocia.
    # Ukrainian
    "Яка столиця України?", # What is the capital of Ukraine?
    "Напишіть коротку розповідь про кота.", # Write a short story about a cat.
    #Vietnamese
    "Thủ đô của Việt Nam là gì?", # What is the capital of Vietnam?
    "Hãy viết một câu ngắn về Vịnh Hạ Long.", # Please write a short sentence about Ha Long Bay.
    #Persian
    "پایتخت ایران کجاست؟", #Where is the capital of Iran?
    "یک جمله کوتاه درباره حافظ بنویسید", #Write a short sentence about Hafez
]

for prompt in test_prompts_multilingual:
    print(f"\n--- Model Output Before Fine-tuning for prompt: {prompt} ---") 
    print(generate_text(template.format(instruction=prompt, response=""), gemma_lm))
    print("\n")


--- Model Output Before Fine-tuning for prompt: What's a fun fact about the Grand Canyon? ---


Instruction:
What's a fun fact about the Grand Canyon?

Response:
It's the Grand Canyon.

What's a fact about the Grand Canyon?

Response:
It's the Grand Canyon.

Grand Canyon:
It's a long word with 7 letters.

Grand Canyon:
Grand Canyon is a national park.

Grand Canyon:
Grand Canyon is a national park.



--- Model Output Before Fine-tuning for prompt: Write a short story about a talking dog. ---


Instruction:
Write a short story about a talking dog.

Response:
The dog was the owner's dog. The dog was very obedient. One day, the owner left the house for two days, and the dog waited for him to return. He waited for his master. After two days, the owner returned and he was very happy and took his dog for a walk. They were in the park, sitting with other people's dogs. When the owner was sitting next to his dog. When he got up to walk his dog for a walk, when his dog started barking. The owner didn't know who was barking at his dog's barking, so he went back to the dog. He took my dog's collar and walked. The owner took his dog for a walk. After a while, the owner came back and the dog still barked. He took his collar off and sat down. Then the dog's voice came from the dog's mouth, "I am sorry for barking at you, sir. I was only trying to keep you safe while you walk my dog" The owner was happy. He gave his dog a pat. And the owner and his dog were walking in peace.



--- Model O

Instruction:
ما هي أقدم مدينة في العالم؟

Response:
A. Babylon
B. Jericho
C. Jerusalem
D. Ur of the Chaldees


Explanation:
Answer C is right.
The Holy City Jerusalem is the oldest and most sacred city in the world. It is the capital of Palestine and was the capital and center for the ancient Israelites.

Instruction:
ما هي أقدم كنيسة في العالم؟

Response:
A. Hagia Sophia in Istanbul.
B. Hagia Sophia in Greece.
C. Santa Sofia in Istanbul.
D. Santa Sofia in Greece.


Explanation:
Answer D is right.
Santa Sofía (in Greek: Αγία Σοφία, lit. "Holy Wisdom") in Istanbul is the largest cathedral in the world. It was built between 532-537 AD. It is the only Christian building from the Roman period to still be in use today.
The church was built on the orders of the Byzantine emperor, Justinian, in the place where the Hagia Sophia had been built under Emperor Theodosius in A.D. 360 as a Christian basilica.

Instruction:
أين هي أكبر كنيسة في العالم؟

Response:
A. Hagia Sophia Istanbul, Turkey.
B. 

Instruction:
اكتب جملة عن أهمية القراءة.

Response:
The importance of reading has an important role in the development of a child. The child, when he starts reading, his thinking and memory develop. The reader is also able to make good use of knowledge.

It is also beneficial for the development of his character. The person who is able to read is able to make use of his own intellect while reading and it has great value for him.

It also enhances his knowledge of a person, he can easily get rid of his ignorance. A good reader can get an idea of ​​the current affairs by reading current affairs newspapers. Reading newspapers is very beneficial in terms of developing a person's vocabulary by reading good books.

A person who reads regularly develops his thinking power, his ability to solve problems is also enhanced.



--- Model Output Before Fine-tuning for prompt: 长城有多长？ ---


...

Instruction:
Напишіть коротку розповідь про кота.

Response:
My cat is a little animal. I have a cat named "Sanya." She is small. Sanya is white-gray in color. She also has black spots. My cat has a round nose, long ears, and a long tail. The eyes are black and round, and the pupils are black. She is very beautiful and gentle by nature.



--- Model Output Before Fine-tuning for prompt: Thủ đô của Việt Nam là gì? ---


Instruction:
Thủ đô của Việt Nam là gì?

Response:
Việt Nam là tên thường gọi của nước Việt Nam. Thủ đô là nơi công điện chính và là nơi đóng chính của các cơ quan hành chính, tổ chức, ngành công nghiệp, xã hội... Thủ đô là nơi tập trung đông đảo nhất trong cả nước về kinh tế, chính trị, văn hóa... Thủ đô là một đặc tọa có một trong năm vị trí quan trọng nhất trong nền kinh tế, xã hội và quân sự của đất nước.


Mỗi nước và từng vùng miền trên thế giới đều có một thủ đô là nơi quan trọng, mang trọng trách lớn trong hệ thống quản lí, điều hành của nước nhà; là nơi tập trung đông đảo nhất các cơ quan, tổ chức, các ngành công nghiệp, xã hội...

Đến nay, thủ đô trên thế giới có khoảng hơn 800 và 90% các nước đều có Thủ đô riêng. Thủ đô là một điểm đặc biệt quan trọng mà các nước luôn chú ý và coi trọng.

Trên toàn thế giới, có rất nhiều quốc gia và địa phương đều có Thủ đô. Thủ đô là nơi tập trung của các ngành công nghiệp, kinh tế; các cơ quan, tổ chức chính phủ, xã hội,... là nơi có các t

Instruction:
Hãy viết một câu ngắn về Vịnh Hạ Long.

Response:
I love the view of Lan Ha Bay. It is the best of all. It is located at Cat Ba Island and there are so many beautiful islands which are all covered with forests, mountains and the waters around. It is such a beautiful place and there are many things for you to do there. You can do fishing, swimming, hiking, sightseeing and also visit historical place, like the cave of Ngoc Hoa, the caves at Nam and the cave where the people have lived for a long time. You will get a chance to take a ferry or a boat to go around in the bay and there will always be a beautiful view for you to behold. It is definitely a nice place to visit in Halong Bay and it is the best one for me.

<b><i>#I_hope_this_helps</i></b>



--- Model Output Before Fine-tuning for prompt: پایتخت ایران کجاست؟ ---


Instruction:
پایتخت ایران کجاست؟

Response:
 Tehran

Explanation:
 

This is the capital of the country. The capital of Iran is Tehran.

Answer :

1. The capital of Iran is Tehran.

Explanation :

This is the capital of the country. The capital of Iran is Tehran. The city is located on the north-south highway connecting the capital to the Gulf of Oman at Bandar Anzali, a port city on the Caspian Sea. Tehran covers an area of ​​4,988 square kilometers, of which the population of the city is 6.9 million.

The city also has a number of important universities.

Answer:

Explanation:

The capital of Iran is Tehran.

Answer:

The capital city of Iran is Tehran.

Explanation:

Hope this helps u ✌️

The capital of Iran is Tehran.

Answer:

Tehran is located on the north south highway and connects to the Gulf of Oman.

Explanation:

Tehran is the capital city of the Islamic Republic of Iran

Answer:

The capital city of India is New Delhi

Explanation:

I think this is the answer because New De

Instruction:
یک جمله کوتاه درباره حافظ بنویسید

Response:
In the 4thcentury BC, a Persian poet by name of Hafiz was born. Hafiz was a very good poet, he was a mystic and he was also a lover of beauty.
The main theme of his poetry is love of life. He was also very good at using Persian language.




## Step 7: Fine-Tuning the Gemma Model with LoRA
We apply LoRA to enable efficient parameter updates during fine-tuning. Key configurations include:
- Optimizer: AdamW with weight decay for transformer models.
- Metrics: Sparse Categorical Accuracy.
- LoRA Rank: Defines the dimensionality of updates.

We use Weights & Biases to monitor training progress and metrics.


In [14]:
LoRA_rank = 8 # you can modify this 
# Enable LoRA for the model and set the LoRA rank to 2,4,...
gemma_lm.backbone.enable_lora(rank=LoRA_rank)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,620,199,168 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,620,199,168 (9.76 GB)

 Trainable params: 5,857,280 (22.34 MB)

 Non-trainable params: 2,614,341,888 (9.74 GB)

In [15]:
gemma_lm.preprocessor.sequence_length = 512
# Using AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.02,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

configs = dict(
    shuffle_buffer = 1000,
    batch_size = 4,
    learning_rate = 5e-5,
    weight_decay = 0.02,
    sequence_length = 512,
    epochs = 16
)

wandb.init(project = "fine-tuning-gemma2_2b_Polyglot",
    config=configs
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


wandb: Currently logged in as: this-is-the-way-2005 (this-is-the-way-2005-independent). Use `wandb login --relogin` to force relogin


wandb: Tracking run with wandb version 0.19.1


wandb: Run data is saved locally in /kaggle/working/wandb/run-20250104_233120-lb6tfv6f
wandb: Run `wandb offline` to turn off syncing.


wandb: Syncing run cool-galaxy-1


wandb: ⭐️ View project at https://wandb.ai/this-is-the-way-2005-independent/fine-tuning-gemma2_2b_Polyglot


wandb: 🚀 View run at https://wandb.ai/this-is-the-way-2005-independent/fine-tuning-gemma2_2b_Polyglot/runs/lb6tfv6f


### Step 8: Training the gemma model:
we train the gemma language model on our ```train_data``` and evaluate it on our ```val_data```, to save time and computation lets use small epochs like 20, If you have more time and computation available, go ahead and increase this!

In [16]:
# Fit the model
history = gemma_lm.fit(train_data, validation_data=val_data, epochs=12, verbose=0, callbacks=[WandbMetricsLogger()])
print("Training finished....")

Training finished....


## Step 9: Evaluate Model Performance After Fine-Tuning
Finally, evaluate the fine-tuned model using the same prompts as earlier. Compare the responses to assess improvements in quality and relevance.


In [18]:
test_prompts_multilingual = [
    # English (American)
    "What's a fun fact about the Grand Canyon?",  # US English
    "Write a short story about a talking dog.", # US English
    # Arabic (Modern Standard)
    "ما هي أقدم مدينة في العالم؟", # What is the oldest city in the world?
    "اكتب جملة عن أهمية القراءة.", # Write a sentence about the importance of reading.
    # Chinese (Simplified)
    "长城有多长？", # How long is the Great Wall?
    "写一个关于猫的短篇故事。", # Write a short story about a cat.
    # Dutch
    "Wat is de hoofdstad van Nederland?", # What is the capital of the Netherlands?
    "Schrijf een korte beschrijving van een molen.", # Write a short description of a windmill.
    # French (European)
    "Quelle est la capitale de la France ?", # What is the capital of France?
    "Écris une courte description de la Tour Eiffel.", # Write a short description of the Eiffel Tower.
    # German
    "Was ist die Hauptstadt von Deutschland?", # What is the capital of Germany?
    "Schreibe eine kurze Beschreibung des Brandenburger Tors.", # Write a short description of the Brandenburg Gate.
    # Italian
    "Qual è la capitale d'Italia?", # What is the capital of Italy?
    "Scrivi una breve descrizione del Colosseo.", # Write a short description of the Colosseum.
    # Japanese
    "日本の首都はどこですか？", # What is the capital of Japan?
    "桜について短い文章を書いてください。", # Please write a short sentence about cherry blossoms.
    # Korean
    "한국의 수도는 어디입니까?", # What is the capital of Korea?
    "한국 음식에 대해 간단히 설명해 주세요.", # Please briefly explain about Korean food.
    # Polish
    "Jaka jest stolica Polski?", # What is the capital of Poland?
    "Napisz krótkie opowiadanie o smoku.", # Write a short story about a dragon.
    # Portuguese (Brazilian)
    "Qual é a capital do Brasil?", # What is the capital of Brazil?
    "Escreva uma breve descrição do Cristo Redentor.", # Write a short description of Christ the Redeemer.
    # Russian
    "Какая столица России?", # What is the capital of Russia?
    "Напишите короткий рассказ о медведе.", # Write a short story about a bear.
    # Spanish (European)
    "¿Cuál es la capital de España?", # What is the capital of Spain?
    "Escribe una breve descripción de la Sagrada Familia.", # Write a short description of the Sagrada Familia.
    # Thai
    "ประเทศไทยมีเมืองหลวงชื่ออะไร", # What is the capital of Thailand?
    "เขียนประโยคสั้นๆ เกี่ยวกับวัดไทย", # Write a short sentence about Thai temples.
    # Turkish
    "Türkiye'nin başkenti neresidir?", # What is the capital of Turkey?
    "Kapadokya hakkında kısa bir açıklama yazın.", # Write a short description about Cappadocia.
    # Ukrainian
    "Яка столиця України?", # What is the capital of Ukraine?
    "Напишіть коротку розповідь про кота.", # Write a short story about a cat.
    #Vietnamese
    "Thủ đô của Việt Nam là gì?", # What is the capital of Vietnam?
    "Hãy viết một câu ngắn về Vịnh Hạ Long.", # Please write a short sentence about Ha Long Bay.
    #Persian
    "پایتخت ایران کجاست؟", #Where is the capital of Iran?
    "یک جمله کوتاه درباره حافظ بنویسید", #Write a short sentence about Hafez the iranian poet 
]

for prompt in test_prompts_multilingual:
    print(f"\n--- Model Output After Fine-tuning for prompt: {prompt} ---") 
    print(generate_text(template.format(instruction=prompt, response=""), gemma_lm))
    print("\n")


--- Model Output After Fine-tuning for prompt: What's a fun fact about the Grand Canyon? ---


Instruction:
What's a fun fact about the Grand Canyon?

Response:
1. The Grand Canyon formed over 5 million years ago when the Colorado River carved a valley in the soft layers of limestone and granite.
2. The Grand Canyon is a natural wonder of the world with a depth of 1,866 meters (6,123 ft.) and a width of 446 meters (~1,460 ft.) at its widest.
3. The canyon can be found in the northwestern corner of Arizona, in the United States.





--- Model Output After Fine-tuning for prompt: Write a short story about a talking dog. ---


Instruction:
Write a short story about a talking dog.

Response:
1. 2. - 3. 4. - 5. 6.
1. 2. 1. 2. 1. 2. 1. 2. 1. 2. 1. 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2
1. 2. 1. 2. 1. 2. 1. 2. 1. 2. 1. 2. 1. 2. 1 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2
1. 2. 1. 2. 1. 2. 1. 2. 1. 2. 1. 2. 1. 2. 1 1 2. 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2
1. 2. 1. 2. 1. 2. 1. 2. 1. 2. 1. 2. 1. 2. 1 1 2. 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2
1. 2. 1. 2. 1. 2. 1. 2. 1. 2. 





--- Model Output After Fine-tuning for prompt: ما هي أقدم مدينة في العالم؟ ---


...

Instruction:
Hãy viết một câu ngắn về Vịnh Hạ Long.

Response:
Bạn đang xem: Viết một câu ngắn về Vịnh Hạ Long
Viết một câu ngắn về Vịnh Hạ Long. 1. Vịnh Hạ Long là một nơi rất đẹp. Cư dân trên thế giới muốn đến đây để du lịch và nghỉ ngơi. Cũng như một số nơi khác trên thế giới, Hạ Long đẹp vì được trời phú hình thành. Mặc dù vậy, con người cũng góp phần tạo nên những vẻ đẹp của Vịnh Hạ Long. Bên cạnh vẻ đẹp thiên nhiên, con người còn sáng tạo ra những nét văn hóa. Đặc biệt là vẻ đẹp trong lòng người.
2. Những nơi như Hạ Long rất đẹp vì được trời phú hình thành. Nhưng cũng vì vậy mà những nơi ấy lại có những vẻ đẹp rất riêng, riêng người nơi ấy. Những người dân trong vùng có nét đẹp riêng của mình vì cuộc sống nơi đó rất vất vả. Vì con người nơi ấy phải đấu tranh với thiên nhiên và cuộc sống mới phát triển trong một thời gian rất dài, nên con người vùng ấy trở nên rất kiên nhẫn và chịu khó.
3. Vịnh Hạ Long đẹp vì đã tạo hình bởi trời đất. Nhưng cũng bởi vậy, Vịnh đã trở nên rất đặc bi



--- Model Output After Fine-tuning for prompt: پایتخت ایران کجاست؟ ---


Instruction:
پایتخت ایران کجاست؟

Response:
کشور ایران و استانهایی که متعلق به آن هستند:
ایران،استان:آذربایجان‌غربی
ایران،استان:آذربایجان‌شرقی
ایران،استان:فارس
ایران،استان:خوزیستان
ایران،استان:کُرمن
ایران،استان:مازندران
ایران،استان:سیستان و بلوچستان
ایران،استان:سغния
ایران،استان:خوزستان
ایران،استان:قزوین


--- Model Output After Fine-tuning for prompt: یک جمله کوتاه درباره حافظ بنویسید ---


Instruction:
یک جمله کوتاه درباره حافظ بنویسید

Response:
آدمی‌های مختلف هر کدام دارای استعدادهای منحصر به فرد خود هستند. استعداد حافظ حافظ است، در حالی که حافظ حافظ است!.
پیشنهاد میکنیم در این زمینه تحقیقات کنید و به صورت اختصاصی و اختصاصی مطلب خود را تدوین نمایید.

#### If you look into our examples and compare it, you can see the models generation has improved for our target language.
Note: since this is a fine-tuned model of a base gemma model and used plain text in 54 languages, we can expect some randomness and other things from its answers, as it has not been fine-tuned on instruct datasets(We will look into this in the next phase).

Also one important thing is now the model is generating the response in the same language as users promts which is a very good imporvment!

### Step 10: Uploading the fine-tuned model to kaggle:
Here we upload the final fine-tuned model to kaggle models so every one can use it!.
we use /kaggle/tmp to save the model, as the model size is larger than kaggle notebooks output directory size.

In [19]:
tmp_model_dir = "/kaggle/tmp/gemma2_2b_Polyglot"  # Use /kaggle/tmp
preset_dir = "gemma2_2b_Polyglot"
os.makedirs(tmp_model_dir, exist_ok=True)
gemma_lm.save_to_preset(tmp_model_dir)

print(f"Model saved to: {tmp_model_dir}")

Model saved to: /kaggle/tmp/gemma2_2b_Polyglot

In [ ]:
import kagglehub
import keras_hub
if "KAGGLE_USERNAME" not in os.environ or "KAGGLE_KEY" not in os.environ:
    kagglehub.login()

model_version = 1
kaggle_username = kagglehub.whoami()["username"]
kaggle_uri = f"kaggle://{kaggle_username}/gemma2/keras/{preset_dir}"
keras_hub.upload_preset(kaggle_uri, tmp_model_dir)
print("Done!")

# Inference
Here we talk about how we can load the fine-tuned model from kaggle and use it:

**For inference we just need to load the fine-tuned model from kaggle to our notebook in the following way:**

for more info check out [here](https://keras.io/api/keras_nlp/models/gemma/gemma_causal_lm/)

specificly:

A preset is a directory of configs, weights and other file assets used to save and load a pre-trained model. The preset can be passed as one of:
* 1. 
a built-in preset identifier like 'bert_base_e
* 2. '
a Kaggle Models handle like 'kaggle://user/bert/keras/bert_base_
* 3. n'
a Hugging Face handle like 'hf://user/bert_base
* 4. en'
a path to a local preset directory like './bert_base_en'

**Infrence step by step:**
* 1. Load the fine-tuned model from kaggle models
* 2. After the model is succesfuly loaded, You can use it to generate text in the targeted language
* Good luck:)

In [ ]:
final_model_id = "kaggle://mahdiseddigh/gemma2/keras/gemma2_2b_Polyglot"
finetuned_gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset(final_model_id)
finetuned_gemma_lm.summary()

In [ ]:
test_prompt = #define your prompt...
print("\n--- Fine-tuned Models Output ---")
print(generate_text(template.format(instruction=test_prompt, response=""), finetuned_gemma_lm))

# Conclusion
This notebook showcased the complete workflow for fine-tuning the Gemma model for mulltilingual text generation. We highlighted:
- Dataset preparation
- Model architecture and parallelism
- Fine-tuning with LoRA
- Performance evaluation pre- and post-training
